In [4]:
import glob
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'  # 也可以使用 tensorflow
# os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,exception_verbosity=high'
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cuda,optimizer=fast_compile'
os.environ["CUDA_VISIBLE_DEVICES"] = "8"

import keras.backend as K

K.set_image_data_format('channels_last')

import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, LSTM, Concatenate, Dense, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten, UpSampling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.datasets import mnist
from urllib.request import urlretrieve
from keras.optimizers import RMSprop, SGD, Adam, sgd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.models import model_from_json

# from read_data import *
a = %pwd
print(a)

%run /root/jupyter/inspace/sang-min/myo_proejct/load_data.py import DataLoader
#%load load_data import DataLoader
#from load_data import DataLoader


'''
Model structure

Input : (100) Vector
Output : (64, 64, 1) image

Generator
100 -> 256
256 -> 16 x 16
16 x 16 -> 32 x 32
32 x 32 -> 64 x 64
64 x 64 -> 128 x 128

Discriminator
128 x 128 -> 64 x 64
64 x 64 -> 32 x 32
32 x 32 -> 16 x 16
16 x 16 -> 8 x 8
8 x 8 -> 64
64 -> 32 (16, 10)

'''
loader = DataLoader(data_path='./dataset_2018_05_16/', is_real_image=False)

conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02)

def toggle_trainable(network, state):
    network.trainable = state
    
    for layer in network.layers:
        layer.trainable = state

def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)

    return model


def generative_model(noise_size, image_channel):
    # lstm_layer = LSTM(80, input_shape=lstm_size)(lstm_input)
    # _ = Concatenate(axis=-1)([lstm_layer, noise_input])
    print(" _ : ", noise_input)
    _ = Dense(2*2*512, input_shape=(100,), activation='relu', bias_initializer='glorot_normal',
              kernel_initializer='glorot_normal')(noise_input)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Reshape((2, 2, 512), input_shape=(2*2*512,))(_)

    _ = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=64, kernel_size=3, padding='same',  kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    #_ = Dropout(0.3)(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=32, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=16, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    #_ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=8, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=image_channel, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = Activation(activation='tanh')(_)

    return Model(inputs=noise_input, outputs=_)


def discriminative_model(image_size, image_channel):
    _ = inputs = Input(shape=(image_size, image_size, image_channel))

    _ = Conv2D(filters=32, kernel_size=(5, 5), border_mode='same', input_shape=(image_size,image_size, image_channel),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=64,kernel_size=(5, 5), border_mode='same', input_shape=(64, 64, 32),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(5, 5), border_mode='same', input_shape=(32, 32, 64),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=256, kernel_size=(5, 5), border_mode='same', input_shape=(16, 16, 128),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=512, kernel_size=(5, 5), border_mode='same', input_shape=(8, 8, 256),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=1, kernel_size=(2, 2), strides=1, padding='same', input_shape=(4, 4, 512),
               kernel_initializer=conv_init)(_)
    # _ = LeakyReLU(alpha=0.2)(_)

    outputs = Flatten()(_)
    outputs = Dense(1, activation='sigmoid')(outputs)

    return Model(inputs=inputs, outputs=outputs)


lstm_size = (300, 16)
noise_size = 100
image_size = 128
input_size = 100
image_channel = 1
learning_rate = 2e-4

discriminator_optim = sgd(lr=0.01, momentum=0.9, nesterov=True)
generator_optim = Adam(lr=1e-3)

# lstm_input = Input(shape=lstm_size)
noise_input = Input(shape=(noise_size,))
real_image = Input(shape=(image_size, image_size, image_channel))

net_g = generative_model(noise_size, image_channel)
net_g.compile(loss='binary_crossentropy', optimizer=generator_optim)
net_g.summary()

net_d = discriminative_model(image_size, image_channel)
net_d.compile(loss='binary_crossentropy', optimizer=discriminator_optim)
net_d.summary()

GAN = Sequential()
GAN.add(net_g)
GAN.add(net_d)
GAN_optim = Adam(lr=1e-3)
GAN.compile(loss='binary_crossentropy', optimizer=GAN_optim)
GAN.summary()


# net_g.compile(loss='binary_crossentropy', optimizer='SGD')


'''
loader = DataLoader(data_path='./MYO_Dataset_label/')

emg = loader.load_emg_data()
image, label = loader.load_image()

print(emg.shape)
print(image.shape, label)

emg = loader.get_emg_datas(10)
images, labels = loader.get_images(10)
print(emg.shape, images.shape, labels.shape)

'''

epoch = 10000
i = 0
time_0 = time.time()
batch_size = 32

d_loss_history = []
g_loss_history = []
d_acc_history = []


while i <= epoch:
    # x_train = loader.get_emg_datas(batch_size)
    images = loader.get_images(batch_size)
    noise = np.random.normal(loc=0.0, scale=1.0,size=[batch_size, noise_size])

    discrim_current_orig_train = images.copy()
    discrim_current_noise =noise.copy()
    discrim_generated_train = net_g.predict(discrim_current_noise)

    # print("Starting to train the discriminator!")
    discrim_current_train = discrim_current_orig_train.copy()
    discrim_current_labels = np.zeros(shape=[batch_size])
    
    # Original data samples have a label 1
    for ix in range(batch_size):
        if np.random.uniform(0.0, 1.0) < 0.95:
            discrim_current_labels[ix] = np.random.uniform(0.7, 1.2)
        else:
            discrim_current_labels[ix] = np.random.uniform(0.0, 0.3)
        
    discriminator_loss_cur = net_d.train_on_batch(discrim_current_train, discrim_current_labels)

    discrim_current_train = discrim_generated_train.copy()
    discrim_current_labels = np.zeros(shape=[batch_size])
    
    # Generated samples have a label 0
    for ix in range(batch_size):
        if np.random.uniform(0.0, 1.0) < 0.95:
            discrim_current_labels[ix] = np.random.uniform(0.0, 0.3)
        else:
            discrim_current_labels[ix] = np.random.uniform(0.7, 1.2)
    discriminator_loss_cur += net_d.train_on_batch(discrim_current_train, discrim_current_labels)
        
    d_loss_history.append(discriminator_loss_cur)
    #d_acc_history.append(100 * d_loss[1])

    toggle_trainable(net_d, False)
    noise2 = np.random.normal(loc=0.0, scale=1.0, size=[batch_size*2, 100])
    gen_current_train = noise2.copy()
    gen_current_labels = np.zeros(shape=[batch_size * 2])
    # When we train the generator we want it to fool the discriminator so we use the opposite labels
    # We use gen_current_labels[:] = 1 instead of using gen_current_labels[:] = 0
    for ix in range(batch_size*2):
        gen_current_labels[ix] = np.random.uniform(0.7, 1.2)
    #print("Starting to train the generator!")
    generator_loss_cur = GAN.train_on_batch(gen_current_train, gen_current_labels)
    g_loss_history.append(generator_loss_cur)
    toggle_trainable(net_d, True)

    print("%d [D loss: %f] [G loss: %f]" % (i,discriminator_loss_cur, generator_loss_cur))
    
    if i % 500 == 0:
        gan_image = net_g.predict(noise)
        print("gan imaga2 : ", gan_image[0].shape)
        cv2.imwrite('./output_image3/' + 'fake_image'+ str(i) + '.png', gan_image[0] * 127.5)
        #cv2.imwrite('./output_image3/' + 'real_image'+ str(i) + '.png', images[0] * 127.5)
        
    i +=1;

net_g.save_weights("./output_image3/g_model.h5")
net_d.save_weights("./output_image3/d_model.h5")
combined_model.save_weights("./output_image3/combined_model.h5")

g_model_json = net_g.to_json()
with open("./output_image3/g_model.json", "w") as json_file:
    json_file.write(g_model_json)
    
d_model_json = net_d.to_json()
with open("./output_image3/d_model.json", "w") as json_file:
    json_file.write(d_model_json)

combined_model_json = combined_model.to_json()
with open("./output_image3/combined_model.json", "w") as json_file:
    json_file.write(combined_model_json)

print("Saved model to disk")

#print("save model ")
plt.figure(1, figsize=(16, 8))
plt.plot(d_loss_history)
plt.ylabel('d_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image3/d_loss_history.png')

#plt.show()

plt.figure(2, figsize=(16, 8))
plt.plot(g_loss_history)
plt.ylabel('g_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image3/g_loss_history.png')

#plt.show()\

plt.figure(3, figsize=(16, 8))
plt.plot(d_acc_history)
plt.ylabel('d_acc')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image3/d_acc_history.png')

plt.show()

print("Finish")

/root/jupyter/inspace/sang-min/myo_proejct
30
./dataset_2018_05_16/1/
15
 _ :  Tensor("input_7:0", shape=(?, 100), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2048)              206848    
_________________________________________________________________
batch_normalization_25 (Batc (None, 2048)              8192      
_________________________________________________________________
reshape_3 (Reshape)          (None, 2, 2, 512)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 2, 2, 256)         1179904   
_________________________________________________________________
batch_normalization_26 (Batc (None, 2, 2, 256)         8         
____________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:133: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(input_shape=(128, 128,..., filters=32, kernel_initializer=<keras.ini..., padding="same", kernel_size=(5, 5))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:134: UserWarning: Update your `AveragePooling2D` call to the Keras 2 API: `AveragePooling2D(padding="valid", pool_size=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:139: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(input_shape=(64, 64, 3..., filters=64, kernel_initializer=<keras.ini..., padding="same", kernel_size=(5, 5))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:140: UserWarning: Update your `AveragePooling2D` call to the Keras 2 API: `AveragePooling2D(padding="valid", pool_size=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:145: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(inp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 128, 128, 32)      832       
_________________________________________________________________
average_pooling2d_11 (Averag (None, 64, 64, 32)        0         
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization_32 (Batc (None, 64, 64, 32)        256       
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 64, 64, 64)        51264     
_________________________________________________________________
average_pooling2d_12 (Averag (None, 32, 32, 64)        0         
__________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 3.723963] [G loss: 2.961917]
gan imaga2 :  (128, 128, 1)
1 [D loss: 2.444046] [G loss: 4.407784]
2 [D loss: 3.709734] [G loss: 9.071479]
3 [D loss: 5.467647] [G loss: 15.198322]
4 [D loss: 5.161254] [G loss: 12.237347]
5 [D loss: 7.687214] [G loss: 9.965475]
6 [D loss: 2.522442] [G loss: 12.842836]
7 [D loss: 16.030071] [G loss: 9.998827]
8 [D loss: 16.696527] [G loss: 6.228770]
9 [D loss: 16.844757] [G loss: 13.503104]
10 [D loss: 18.226791] [G loss: 15.052586]
11 [D loss: 14.284543] [G loss: 0.941207]
12 [D loss: 15.091835] [G loss: 0.535742]
13 [D loss: 14.888617] [G loss: 1.990538]
14 [D loss: 14.453547] [G loss: 9.987333]
15 [D loss: 12.568217] [G loss: 6.345208]
16 [D loss: 13.540942] [G loss: 2.119086]
17 [D loss: 12.544095] [G loss: 1.522686]
18 [D loss: 13.587290] [G loss: 0.176606]
19 [D loss: 14.908887] [G loss: 1.242170]
20 [D loss: 13.249952] [G loss: 0.602819]
21 [D loss: 13.874027] [G loss: 0.309603]
22 [D loss: 14.327305] [G loss: 1.446420]
23 [D loss: 15.542

...

940 [D loss: 14.051367] [G loss: 0.902074]
941 [D loss: 14.201212] [G loss: 0.538392]
942 [D loss: 13.689951] [G loss: 0.986566]
943 [D loss: 14.459675] [G loss: 0.491899]
944 [D loss: 14.522274] [G loss: 0.992477]
945 [D loss: 14.899453] [G loss: 0.370897]
946 [D loss: 14.799355] [G loss: 0.873489]
947 [D loss: 13.969109] [G loss: 0.874276]
948 [D loss: 14.902170] [G loss: 1.145006]
949 [D loss: 11.836753] [G loss: 0.871283]
950 [D loss: 15.401567] [G loss: 0.747339]
951 [D loss: 12.235869] [G loss: 0.928475]
952 [D loss: 14.780125] [G loss: 1.155107]
953 [D loss: 14.492407] [G loss: 0.569915]
954 [D loss: 13.006274] [G loss: 0.577412]
./dataset_2018_05_16/5/
955 [D loss: 15.439568] [G loss: 1.402355]
956 [D loss: 11.844077] [G loss: 0.879253]
957 [D loss: 14.016349] [G loss: 0.859386]
958 [D loss: 14.335340] [G loss: 0.743413]
959 [D loss: 13.852217] [G loss: 0.724884]
960 [D loss: 15.255304] [G loss: 0.437181]
961 [D loss: 14.459753] [G loss: 1.039362]
962 [D loss: 12.954836] [G los

1124 [D loss: 13.859226] [G loss: 0.468996]
1125 [D loss: 16.124920] [G loss: 1.178203]
1126 [D loss: 14.775661] [G loss: 0.759353]
1127 [D loss: 14.110192] [G loss: 1.021067]
1128 [D loss: 14.173207] [G loss: 0.712400]
1129 [D loss: 14.542517] [G loss: 0.574353]
1130 [D loss: 15.020832] [G loss: 1.084302]
1131 [D loss: 14.114505] [G loss: 0.716575]
1132 [D loss: 12.901185] [G loss: 1.298359]
1133 [D loss: 13.648646] [G loss: 0.978499]
1134 [D loss: 14.144199] [G loss: 0.535538]
1135 [D loss: 14.605224] [G loss: 0.725251]
1136 [D loss: 14.756900] [G loss: 0.642934]
1137 [D loss: 13.376423] [G loss: 0.547259]
1138 [D loss: 14.209604] [G loss: 0.930685]
1139 [D loss: 13.852098] [G loss: 1.044466]
1140 [D loss: 13.989395] [G loss: 1.110661]
1141 [D loss: 13.424279] [G loss: 0.739418]
1142 [D loss: 13.258265] [G loss: 1.590565]
1143 [D loss: 15.671098] [G loss: 0.562435]
1144 [D loss: 16.416231] [G loss: 0.938915]
1145 [D loss: 13.847425] [G loss: 0.768272]
1146 [D loss: 14.768942] [G loss

1308 [D loss: 15.050111] [G loss: 0.629135]
1309 [D loss: 13.669785] [G loss: 0.606735]
1310 [D loss: 15.151925] [G loss: 0.912833]
1311 [D loss: 15.508574] [G loss: 0.721542]
1312 [D loss: 14.432536] [G loss: 0.567471]
1313 [D loss: 15.718801] [G loss: 1.334877]
1314 [D loss: 14.710875] [G loss: 1.088668]
1315 [D loss: 12.645543] [G loss: 0.490857]
1316 [D loss: 13.938638] [G loss: 0.887567]
1317 [D loss: 14.690598] [G loss: 1.070586]
1318 [D loss: 14.081528] [G loss: 0.845867]
1319 [D loss: 13.574107] [G loss: 0.327829]
./dataset_2018_05_16/3/
1320 [D loss: 15.493622] [G loss: 0.790289]
1321 [D loss: 14.944880] [G loss: 1.338596]
1322 [D loss: 15.596830] [G loss: 0.847537]
1323 [D loss: 13.011415] [G loss: 0.129156]
1324 [D loss: 13.418220] [G loss: 1.025440]
1325 [D loss: 15.729980] [G loss: 0.635713]
1326 [D loss: 12.797628] [G loss: 0.789555]
1327 [D loss: 12.334536] [G loss: 0.624423]
1328 [D loss: 14.663158] [G loss: 0.753064]
1329 [D loss: 13.572482] [G loss: 1.064315]
1330 [D 

1492 [D loss: 14.652448] [G loss: 0.502994]
1493 [D loss: 14.268255] [G loss: 0.551621]
1494 [D loss: 14.794157] [G loss: 1.286604]
1495 [D loss: 12.150485] [G loss: 0.747723]
1496 [D loss: 14.652231] [G loss: 0.860239]
1497 [D loss: 14.874874] [G loss: 0.358973]
1498 [D loss: 12.277978] [G loss: 0.941560]
1499 [D loss: 15.868210] [G loss: 0.639853]
1500 [D loss: 15.868701] [G loss: 0.556695]
gan imaga2 :  (128, 128, 1)
1501 [D loss: 14.854440] [G loss: 1.153178]
1502 [D loss: 14.118792] [G loss: 0.879461]
1503 [D loss: 15.127881] [G loss: 1.427886]
1504 [D loss: 13.530153] [G loss: 1.216541]
1505 [D loss: 13.851771] [G loss: 1.121686]
1506 [D loss: 14.557508] [G loss: 0.910924]
1507 [D loss: 15.275071] [G loss: 1.240774]
1508 [D loss: 14.987554] [G loss: 1.062428]
1509 [D loss: 14.514212] [G loss: 1.020890]
1510 [D loss: 14.006155] [G loss: 0.539398]
1511 [D loss: 14.207438] [G loss: 0.521599]
1512 [D loss: 12.906631] [G loss: 1.103849]
1513 [D loss: 13.172073] [G loss: 0.615024]
1514

KeyboardInterrupt: 